# Downloading and processing data for DBHW course

**ST0317**

In [7]:
import dotenv

# load in environment variables
dotenv.load_dotenv()

import datetime as dt
import magicpy.analysis as analysis
import magicpy.datalevel as dl
import magicpy.periods as periods
import magicpy.moons as moons
import magicpy.zeniths as zeniths


In [8]:
mc = analysis.MC(
    period=periods.ST0307,
    zenith=zeniths.za05to35,
)

Loading existing analysis from /remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0307/MC_ST0307_za05to35_None.magicpy
INFO: RFs not found for: ST0307 za05to35 weak moon.
INFO: RFs not found for: ST0307 za05to35 moderate moon.
INFO: RFs not found for: ST0307 za05to35 decent moon.
INFO: RFs not found for: ST0307 za05to35 strong moon.
INFO: RFs not found for: ST0307 za05to35 high moon.
INFO: RFs not found for: ST0307 za05to35 super moon.
INFO: RFs not found for: ST0307 za05to35 reduced_decent moon.
INFO: RFs not found for: ST0307 za05to35 reduced_strong moon.
INFO: RFs not found for: ST0307 za05to35 reduced_high moon.


In [39]:
mc.runs.local_path.iloc[0]

PosixPath('/remote/magicdata5/jgreen/MAGICPY_ANALYSIS/MC/ST0307/za05to35/weak/Calibrated_M1/GA_M1_za05to35_8_821319_Y_w0.root')

In [ ]:
mc.get_data(
    data_level=[dl.Calibrated, dl.SuperStar]
)

# Get Proton files

In [24]:
import os
import certifi
from webdav3.client import Client
from webdav3.exceptions import ConnectionException
from tqdm import tqdm
from pathlib import Path
import magicpy.constants as constants

In [ ]:
output_dir = "/remote/lstdata01/jgreen/MAGIC_MC/protons"

In [ ]:
calibrated_m1_dir = "https://webdav-magic.pic.es:8451/Data/MonteCarlo_Stereo/M1_DRS4_1039_M2_DRS4_1039/M1_AD3.5_MF0.68_M2_AD5.0_MF0.71/proton/za05to35/diffuse_4.0/mmcs699/calibrated_M1/"
calibrated_m2_dir = "https://webdav-magic.pic.es:8451/Data/MonteCarlo_Stereo/M1_DRS4_1039_M2_DRS4_1039/M1_AD3.5_MF0.68_M2_AD5.0_MF0.71/proton/za05to35/diffuse_4.0/mmcs699/calibrated_M2/"
superstar_dir = "https://webdav-magic.pic.es:8451/Data/MonteCarlo_Stereo/M1_DRS4_1039_M2_DRS4_1039/M1_AD3.5_MF0.68_M2_AD5.0_MF0.71/proton/za05to35/diffuse_4.0/mmcs699/superstar/"

In [29]:
# remove URL from the front
calibrated_m1_dir = calibrated_m1_dir = calibrated_m1_dir[len(constants.pic_webdav) :]
calibrated_m2_dir = calibrated_m2_dir = calibrated_m2_dir[len(constants.pic_webdav) :]
superstar_dir = superstar_dir = superstar_dir[len(constants.pic_webdav) :]

output_dir = Path(output_dir).absolute()

# start the webdav connection

pic_username = os.getenv("PIC_USERNAME")
pic_password = os.getenv("PIC_PASSWORD")

dav_options = {
    "webdav_hostname": constants.pic_webdav,
    "webdav_login": pic_username,
    "webdav_password": pic_password,
    "cert_path": certifi.where(),
}

client = Client(dav_options)

In [ ]:
local_files = []

download_dirs = [
    calibrated_m1_dir,
    calibrated_m2_dir,
    superstar_dir,
]

for download_dir in download_dirs:
    
    print(f"Downloading from {download_dir}")
    
    download_list = []

    for file in client.list(download_dir):
        if file.startswith("GA") and file.endswith(".root"):
            download_list.append(f"{download_dir}{file}")
            
    download_list = sorted(download_list)
    
    download_to = output_dir / Path(download_dir).name
    
    if not download_to.exists():
        download_to.mkdir(parents=True)
    
    for file in tqdm(download_list):
        local_file = download_to / Path(file).name
        
        if not local_file.exists():
            client.download_sync(str(file), str(local_file))
            
        local_files.append(local_file)